In [2]:
import fidap, pandas as pd, importlib
importlib.reload(fidap)

<module 'fidap' from 'C:\\Users\\User\\Google Drive\\Python\\notebooks\\fidap.py'>

Let's first explore the data and find some basic numbers here.

In [3]:
fidap.sql("""
    select count(*) as count, sum(transactionvalue) as tvalue, 
    count(distinct(ticker)) as tickers, min(filingdate) from insider_trades
""")

,count,tvalue,tickers,min
0,11901766,9.274782e+12,15104,2005-01-03


We can see these same numbers in the Fidap dashboard as well.

Let's see some of the individual rows here.

In [4]:
fidap.sql("""
    select * from insider_trades where ticker='AAPL' limit 5
""")

,ticker,filingdate,formtype,issuername,ownername,officertitle,isdirector,isofficer,istenpercentowner,transactiondate,...,sharesownedfollowingtransaction,transactionpricepershare,transactionvalue,securitytitle,directorindirect,natureofownership,dateexercisable,priceexercisable,expirationdate,rownum
0,AAPL,2020-10-13,4,APPLE INC,MAESTRI LUCA,Senior Vice President CFO,N,Y,N,2020-10-09,...,110272.0,116.89,28454650.0,Common Stock,D,None,None,None,None,1.0
1,AAPL,2020-10-05,4,APPLE INC,ADAMS KATHERINE L,SVP GC and Secretary,N,Y,N,2020-10-01,...,550892.0,NaN,NaN,Common Stock,D,None,None,None,None,1.0
2,AAPL,2020-10-05,4,APPLE INC,ADAMS KATHERINE L,SVP GC and Secretary,N,Y,N,2020-10-01,...,323396.0,116.79,26569258.0,Common Stock,D,None,None,None,None,2.0
3,AAPL,2020-10-05,4,APPLE INC,ADAMS KATHERINE L,SVP GC and Secretary,N,Y,N,2020-10-01,...,NaN,NaN,NaN,Restricted Stock Unit,D,None,None,None,None,3.0
4,AAPL,2020-10-05,4,APPLE INC,WILLIAMS JEFFREY E,COO,N,Y,N,2020-10-01,...,1008340.0,NaN,NaN,Common Stock,D,None,None,None,None,1.0


We can actually get info on each of the columns from the metadata table. Let's take a look.

In [5]:
fidap.sql("""
    select * from insider_trades where ticker='AAPL' and securitytitle = 'Common Stock' limit 5
""")

,ticker,filingdate,formtype,issuername,ownername,officertitle,isdirector,isofficer,istenpercentowner,transactiondate,...,sharesownedfollowingtransaction,transactionpricepershare,transactionvalue,securitytitle,directorindirect,natureofownership,dateexercisable,priceexercisable,expirationdate,rownum
0,AAPL,2020-10-05,4,APPLE INC,ADAMS KATHERINE L,SVP GC and Secretary,N,Y,N,2020-10-01,...,550892.0,NaN,NaN,Common Stock,D,None,None,None,None,1.0
1,AAPL,2020-10-05,4,APPLE INC,ADAMS KATHERINE L,SVP GC and Secretary,N,Y,N,2020-10-01,...,323396.0,116.79,26569258.0,Common Stock,D,None,None,None,None,2.0
2,AAPL,2020-10-05,4,APPLE INC,WILLIAMS JEFFREY E,COO,N,Y,N,2020-10-01,...,1008340.0,NaN,NaN,Common Stock,D,None,None,None,None,1.0
3,AAPL,2020-10-05,4,APPLE INC,WILLIAMS JEFFREY E,COO,N,Y,N,2020-10-01,...,746603.0,116.79,30568264.0,Common Stock,D,None,None,None,None,2.0
4,AAPL,2020-10-05,4,APPLE INC,WILLIAMS JEFFREY E,COO,N,Y,N,2020-10-02,...,517646.0,113.52,25991198.0,Common Stock,D,None,None,None,None,3.0


In [6]:
# need to link with tickers table
fidap.sql("""
    select ticker, count(*) as count, sum(transactionvalue) as tvalue from insider_trades
    where transactionvalue > 0 group by ticker order by tvalue desc limit 10
""")

,ticker,count,tvalue
0,NGBL,1001,1.425084e+12
1,CDNC,303,2.035737e+11
2,GGP,1325,8.086486e+10
3,PWRM,88,6.180867e+10
4,TELOZ,63,5.091514e+10
5,HLT,361,5.049469e+10
6,OMAG,69,4.371894e+10
7,SOYL,83,4.360486e+10
8,TMRK,191,4.332359e+10
9,QVDX,171,4.293356e+10


In [7]:
fidap.sql("""
    select insider_trades.ticker, insider_trades.filingdate, insider_trades.transactionvalue from insider_trades, tickers
    where insider_trades.ticker = tickers.ticker and insider_trades.transactionvalue > 100000 and insider_trades.securitytitle = 'Common Stock' 
    and tickers.sector = 'Technology'
    limit 100
""")

,ticker,filingdate,transactionvalue
0,AAPL,2020-10-05,26569258.0
1,AAPL,2020-10-05,30568264.0
2,AAPL,2020-10-05,25991198.0
3,AAPL,2020-10-05,3241397.0
4,AAPL,2020-10-05,32193046.0
...,...,...,...
95,ETSY,2020-10-05,252450.0
96,ETSY,2020-10-05,255981.0
97,ETSY,2020-10-05,103337.0
98,ETSY,2020-10-05,286126.0
